In [2]:
from atgreen import *

In [ ]:
PATH=""
BUFFER_SIZE=3000

In [ ]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

### SAVE RASTERS

In [ ]:
for city in cities_list:
    print(city)
    #Geometry for clip
    geometry_to_clip=query4cityboundary(city, db_params, BUFFER_SIZE).geometry
    wcesa2raster(geometry_to_clip, f"{PATH}/esa",True, city)

### GENERATE SQL TABLE ON ALL CITIES AT ONCE

In [ ]:
#Load multiple raster at once    
multiplerast2sql(f"{PATH}/esa" ,f"{PATH}/tmp", "esa", 'tiff', '4326')
sqlRasterTable2db('esa',f"{PATH}/tmp",db_params)

#Move to right schema:
conn = psycopg2.connect(
        dbname=db_params['db_name'], user=db_params['db_user'], password=db_params['db_password'], host=db_params['db_host'])
conn.autocommit=True
conn.cursor().execute(f"""ALTER TABLE esa SET SCHEMA esa;""")
conn.close()

#Remove files
for f in os.listdir(f"{PATH}/esa/"):
    os.remove(f"{PATH}/esa/{f}")